In [1]:
import numpy as np

"""
This file implements various first-order update rules that are commonly used
for training neural networks. Each update rule accepts current weights and the
gradient of the loss with respect to those weights and produces the next set of
weights. Each update rule has the same interface:

def update(w, dw, config=None):

Inputs:
  - w: A numpy array giving the current weights.
  - dw: A numpy array of the same shape as w giving the gradient of the
    loss with respect to w.
  - config: A dictionary containing hyperparameter values such as learning
    rate, momentum, etc. If the update rule requires caching values over many
    iterations, then config will also hold these cached values.

Returns:
  - next_w: The next point after the update.
  - config: The config dictionary to be passed to the next iteration of the
    update rule.

NOTE: For most update rules, the default learning rate will probably not
perform well; however the default values of the other hyperparameters should
work well for a variety of different problems.

For efficiency, update rules may perform in-place updates, mutating w and
setting next_w equal to w.
"""

https://zhuanlan.zhihu.com/p/30034654

In [2]:

def sgd(w, dw, config=None):
    """
    Performs vanilla stochastic gradient descent.

    config format:
    - learning_rate: Scalar learning rate.
    """
    if config is None: config = {}
    config.setdefault('learning_rate', 1e-2)

    w -= config['learning_rate'] * dw
    return w, config


![](img/momentum.png)

In [7]:

def sgd_momentum(w, dw, config=None):
    """
    Performs stochastic gradient descent with momentum.

    config format:
    - learning_rate: Scalar learning rate.
    - momentum: Scalar between 0 and 1 giving the momentum value.
          Setting momentum = 0 reduces to sgd.
    - velocity: A numpy array of the same shape as w and dw used to store a
          moving average of the gradients.
    """
    if config is None: config = {}
    config.setdefault('learning_rate', 1e-2)
    config.setdefault('momentum', 0.9)
    
    v = config.get('velocity', np.zeros_like(w))

    next_w = None
    
    momentum = config.get('momentum')
    learning_rate = config.get('learning_rate')
    
    v = momentum * v + (1 - momentum) * dw
    next_w = w - learning_rate * v
    
    config['velocity'] = v
    
    return next_w, config


$$ S_t = \beta S_{t-1} + (1-\beta)dW^2 $$
$$ W =W-\alpha * \frac{dW}{\sqrt{S_t+\epsilon}} $$ 

$$ \beta = 0.99 $$
$$ \epsilon = 10^{-8} $$


In [4]:

def rmsprop(x, dx, config=None):
    """
    Uses the RMSProp update rule, which uses a moving average of squared
    gradient values to set adaptive per-parameter learning rates.

    config format:
    - learning_rate: Scalar learning rate.
    - decay_rate: Scalar between 0 and 1 giving the decay rate for the squared
      gradient cache.
    - epsilon: Small scalar used for smoothing to avoid dividing by zero.
    - cache: Moving average of second moments of gradients.
    """
    if config is None: config = {}
    config.setdefault('learning_rate', 1e-2)
    config.setdefault('decay_rate', 0.99) 
    config.setdefault('epsilon', 1e-8)
    config.setdefault('cache', np.zeros_like(x))

    next_x = None
    
    decay_rate = config.get('decay_rate')
    cache = config.get('cache')
    learning_rate = config.get('learning_rate')
    epsilon = config.get('epsilon')
    
    config['cache'] = decay_rate * cache + (1 - decay_rate) * (dx**2)
    
    x += - learning_rate * dx / (np.sqrt(config['cache']) + epsilon )
    
    next_x = x
    
    return next_x, config


![](img/adam.png)

In [5]:

def adam(x, dx, config=None):
    """
    Uses the Adam update rule, which incorporates moving averages of both the
    gradient and its square and a bias correction term.

    config format:
    - learning_rate: Scalar learning rate.
    - beta1: Decay rate for moving average of first moment of gradient.
    - beta2: Decay rate for moving average of second moment of gradient.
    - epsilon: Small scalar used for smoothing to avoid dividing by zero.
    - m: Moving average of gradient.
    - v: Moving average of squared gradient.
    - t: Iteration number.
    """
    if config is None: config = {}
    config.setdefault('learning_rate', 1e-3)
    config.setdefault('beta1', 0.9)
    config.setdefault('beta2', 0.999)
    config.setdefault('epsilon', 1e-8)
    config.setdefault('m', np.zeros_like(x))
    config.setdefault('v', np.zeros_like(x))
    config.setdefault('t', 1)

    next_x = None

    # m = beta1*m + (1-beta1)*dx
    # mt = m / (1-beta1**t)
    # v = beta2*v + (1-beta2)*(dx**2)
    # vt = v / (1-beta2**t)
    # x += - learning_rate * mt / (np.sqrt(vt) + eps)
    
    config['t'] += 1
    beta1 = config.get('beta1')
    beta2 = config.get('beta2')
    epsilon = config.get('epsilon')
    learning_rate = config.get('learning_rate')
    t = config.get('t')
    v = config.get('v')

    config['m'] =  beta1 * config['m'] + (1 - beta1) * dx
    mt = config['m'] / (1 - beta1**t)
    v = beta2*v + (1-beta2)*(dx**2)
    vt = v / (1-beta2**t)
    next_x = x - learning_rate * mt / (np.sqrt(vt) + epsilon)
    config['v'] = v

    return next_x, config
